In [16]:
import numpy as np
import pandas as pd
import datetime as dt 
import time, warnings
import matplotlib.pyplot as plt
pd.plotting.scatter_matrix
%matplotlib inline 
import seaborn as sns


In [31]:
retail_df = pd.read_csv("data1.csv", dtype={'CustomerID': str,'InvoiceID' : str })
retail_fr = retail_df[retail_df['Country']=='United Kingdom']

retail_fr = retail_fr[retail_fr['Quantity']>0]
retail_fr = retail_fr[retail_fr['InvoiceDate']>= "2020-12-09"]
retail_fr.dropna(subset=['CustomerID'],how='all', inplace=True)
retail_fr.shape

(176137, 8)

In [32]:

print("Number of transactions: ", retail_fr['InvoiceNo'].nunique())
print("Number of products bought: ",retail_fr['StockCode'].nunique())
print("Number of customers:", retail_fr['CustomerID'].nunique() )


Number of transactions:  8789
Number of products bought:  3294
Number of customers: 2864


In [34]:
#RECENCY 
retail_fr['InvoiceDate'].max()


'9/9/2011 9:52'

In [35]:
#last date we have is 2011-12-09, we shall use it as our refernce

now= dt.date(2011,9,9)

#new column
retail_fr['date'] = pd.DatetimeIndex(retail_fr['InvoiceDate']).date
retail_fr.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
105335,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,3/1/2011 8:30,7.95,14620,United Kingdom,2011-03-01
105336,545220,48194,DOORMAT HEARTS,2,3/1/2011 8:30,7.95,14620,United Kingdom,2011-03-01
105337,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,3/1/2011 8:30,1.65,14620,United Kingdom,2011-03-01
105338,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,3/1/2011 8:30,4.95,14620,United Kingdom,2011-03-01
105339,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,3/1/2011 8:30,3.75,14620,United Kingdom,2011-03-01


In [36]:
recency_df = retail_fr.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurchaseDate']
recency_df.head()

,CustomerID,LastPurchaseDate
0,12747,2011-08-22
1,12748,2011-09-30
2,12749,2011-08-01
3,12820,2011-09-26
4,12821,2011-05-09


In [37]:
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(lambda x: (now - x).days) #calculating recency
recency_df.head()

,CustomerID,LastPurchaseDate,Recency
0,12747,2011-08-22,18
1,12748,2011-09-30,-21
2,12749,2011-08-01,39
3,12820,2011-09-26,-17
4,12821,2011-05-09,123


In [38]:
#drop LastPurshaseDate
recency_df.drop('LastPurchaseDate', axis=1, inplace=True)

In [39]:
#Frequency
#drop duplicates
retail_fr_copy = retail_fr
retail_fr_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)

#frequency of purchases
frequency_df = retail_fr_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID', 'Frequency']
frequency_df.head()

,CustomerID,Frequency
0,12747,5
1,12748,96
2,12749,3
3,12820,1
4,12821,1


In [40]:
#Monetary
#create column total cost
retail_fr['TotalCost'] = retail_fr['Quantity'] * retail_fr['UnitPrice']
monetary_df= retail_fr.groupby(by='CustomerID', as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID', 'Monetary']
monetary_df.head()

,CustomerID,Monetary
0,12747,191.85
1,12748,1054.43
2,12749,67.00
3,12820,15.00
4,12821,19.92


In [41]:
#create RFM table
#merge recency and frequency dataframe
temp_df = recency_df.merge(frequency_df, on='CustomerID')
temp_df.head()

,CustomerID,Recency,Frequency
0,12747,18,5
1,12748,-21,96
2,12749,39,3
3,12820,-17,1
4,12821,123,1


In [42]:
#merge with monetary df
rfm_df = temp_df.merge(monetary_df, on='CustomerID')
rfm_df.set_index('CustomerID', inplace=True)
rfm_df.head()

,Recency,Frequency,Monetary
CustomerID,,,
12747,18,5,191.85
12748,-21,96,1054.43
12749,39,3,67.00
12820,-17,1,15.00
12821,123,1,19.92


In [43]:
#RFM ANALYSY
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,Recency,Frequency,Monetary
0.25,-6.0,1.0,16.35
0.50,28.0,2.0,35.40
0.75,92.0,3.0,92.42


In [47]:
#higher recency is bad but higher monetary and frequency is good
## Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)

def RScore(x,p,d):
  if x <= d[p][0.25]:
    return 4
  elif x <= d[p][0.5]: 
    return 3
  elif x <= d[p][0.75]:   
    return 2 
  else :
    return 1



def FMScore(x,p,d):
  if x <= d[p][0.25]:
    return 1
  elif x <= d[p][0.5]: 
    return 2
  elif x <= d[p][0.75]:   
    return 3 
  else :
    return 4




In [49]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

#rfm score
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)

rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12747,18,5,191.85,3,4,4,344
12748,-21,96,1054.43,4,4,4,444
12749,39,3,67.00,2,3,3,233
12820,-17,1,15.00,4,1,1,411
12821,123,1,19.92,1,1,2,112


In [52]:
#Best score is 444
#analysis

print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers:',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  218
Loyal Customers:  687
Big Spenders:  716
Almost Lost:  52
Lost Customers:  5
Lost Cheap Customers: 278
